In [37]:
import pathlib as pl
import pandas as pd
import collections as col
import math as ma
import hashlib as hl

label_file = pl.Path(
    "/home/ebertp/work/code/marschall-lab/project-male-assembly/data",
    "veritymap_expert-label.tsv"
)

PAR1_SAMPLES = [
    "HG01890", "HC02666", "HC19384", "NA19331",
    "HG02011", "HG03371", "NA19317", "HC18989",
    "HG03009", "HG00358"
]

labels = dict()

with open(label_file, "r") as table:
    for line in table:
        if not line.strip() or line.startswith("contig"):
            continue
        c, s, e = line.strip().split()[:3]
        name = f"{c}.{s}.{e}"
        name_id = hl.md5(name.encode("utf-8")).hexdigest()
        labels[name_id] = name
        
all_bed_files = pl.Path(
    "/home/ebertp/work/projects/sig_chry/paper/data/error_regions/sample_beds"
).glob("*.flagged-all.bed")

tool_counts = col.Counter()
par_flagged = []
for bed_file in all_bed_files:
    sample = bed_file.name.split(".")[0]
    if sample not in PAR1_SAMPLES:
        continue
    with open(bed_file, "r") as table:
        for line in table:
            if not line.strip() or line.startswith("#"):
                continue
            c, s, e, n, tool = line.strip().split()[:5]
            if "PAR1" not in c:
                continue
            if int(e) - int(s) < 2:
                continue
            if int(s) > 2.5e6:
                continue
            par_flagged.append(
                    "\t".join([sample, c, s, e, n, tool])
            )
            name = f"{c}.{s}.{e}"
            name_id = hl.md5(name.encode("utf-8")).hexdigest()
            try:
                known = labels[name_id]
                print("same: ", known)
                print(bed_file.name)
                print('---')
            except KeyError:
                continue

with open("par_regions.tsv", "w") as dump:
    _ = dump.write("\n".join(par_flagged) + "\n")

same:  chrY.01-23.01.PAR1-HET3_Yq.RV.unassigned-0000904.NA19317.322400.326200
NA19317.HIFIRW.ONTUL.na.chrY.flagged-all.bed
---
same:  chrY.01-21.01.PAR1-AMPL7.RV.unassigned-0000811.NA19331.323000.323400
NA19331.HIFIRW.ONTUL.na.chrY.flagged-all.bed
---
same:  chrY.01-21.01.PAR1-AMPL7.RV.unassigned-0000811.NA19331.323800.327000
NA19331.HIFIRW.ONTUL.na.chrY.flagged-all.bed
---
